<a href="https://colab.research.google.com/github/amitesh23s/phrase-to-phrase-matching/blob/dev/US_Patent_Phrase_to_phrase_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import pandas as pd

df = pd.read_csv('/content/train.csv')

df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [53]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [54]:
df.input.head()

,input
0,TEXT1: A47; TEXT2: abatement of pollution; ANC...
1,TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2,TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3,TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4,TEXT1: A47; TEXT2: forest region; ANC1: abatement


In [55]:
!pip install transformers

In [56]:
!pip install -U datasets

In [57]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

In [58]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [59]:
model_nm = 'microsoft/deberta-v3-small'

In [60]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [61]:
tokz.tokenize("G'day folks, I'm Jeremy from fast.ai!")

['▁G',
 "'",
 'day',
 '▁folks',
 ',',
 '▁I',
 "'",
 'm',
 '▁Jeremy',
 '▁from',
 '▁fast',
 '.',
 'ai',
 '!']

In [62]:
ds[0]

{'id': '37d61fd2272659b1',
 'anchor': 'abatement',
 'target': 'abatement of pollution',
 'context': 'A47',
 'score': 0.5,
 'input': 'TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement'}

In [63]:
def tok_func(x): return tokz(x["input"])

In [64]:
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [65]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [66]:
tokz.vocab['▁of']

265

In [67]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [68]:
eval_df = pd.read_csv('/content/test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [69]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [ ]:
dds['train'].head()

In [70]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [71]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [2]:
from transformers import TrainingArguments,Trainer
import numpy as np

In [73]:
bs = 128
epochs = 4
lr = 8e-5

In [74]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [75]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-75-48e257fff1ad>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],


In [76]:
trainer.train();

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.032443,0.796735
2,No log,0.021978,0.824755
3,0.030200,0.021774,0.832089


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.032443,0.796735
2,No log,0.021978,0.824755
3,0.030200,0.021774,0.832089
4,0.030200,0.022139,0.833295


In [77]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.53528708],
       [ 0.7040419 ],
       [ 0.61700708],
       [ 0.37734121],
       [-0.0509688 ],
       [ 0.50003755],
       [ 0.51987237],
       [-0.02898528],
       [ 0.28899884],
       [ 1.08641386],
       [ 0.26049408],
       [ 0.2437789 ],
       [ 0.72636199],
       [ 0.77730274],
       [ 0.74244505],
       [ 0.44167656],
       [ 0.29495919],
       [-0.03121291],
       [ 0.64821547],
       [ 0.34236532],
       [ 0.46266043],
       [ 0.23722067],
       [ 0.16283073],
       [ 0.2087023 ],
       [ 0.57298124],
       [-0.00867584],
       [-0.04895093],
       [-0.0269066 ],
       [-0.03014463],
       [ 0.59012991],
       [ 0.23637071],
       [-0.01594272],
       [ 0.71896541],
       [ 0.47547603],
       [ 0.46531668],
       [ 0.20062666]])

In [78]:
preds = np.clip(preds, 0, 1)
preds

array([[0.53528708],
       [0.7040419 ],
       [0.61700708],
       [0.37734121],
       [0.        ],
       [0.50003755],
       [0.51987237],
       [0.        ],
       [0.28899884],
       [1.        ],
       [0.26049408],
       [0.2437789 ],
       [0.72636199],
       [0.77730274],
       [0.74244505],
       [0.44167656],
       [0.29495919],
       [0.        ],
       [0.64821547],
       [0.34236532],
       [0.46266043],
       [0.23722067],
       [0.16283073],
       [0.2087023 ],
       [0.57298124],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.59012991],
       [0.23637071],
       [0.        ],
       [0.71896541],
       [0.47547603],
       [0.46531668],
       [0.20062666]])

In [3]:
classes = np.array([0.0, 0.25, 0.50, 0.75, 1.0])

preds = classes[np.abs(classes[:, None] - preds[None, :]).argmin(axis=0)]

NameError: name 'preds' is not defined

In [ ]:
true_labels = eval_ds['labels']
accuracy = np.mean(preds == true_labels)

print(f"Accuracy: {accuracy * 100:.2f}%")